In [2]:
source = "./Datasets/Kvasir-SEG/test/images"

In [12]:
from ultralytics import YOLO


# Load a model
model = YOLO("best.pt")  # pretrained YOLO11n model

# Run batched inference on a list of images
#results_yolo = model.predict(source, imgsz=352, conf=0.5)
results_yolo = model(source)  # return a list of Results objects

# Process results list
# for result in results_yolo:
#     boxes = result.boxes  # Boxes object for bounding box outputs
#     masks = result.masks  # Masks object for segmentation masks outputs
#     keypoints = result.keypoints  # Keypoints object for pose outputs
#     probs = result.probs  # Probs object for classification outputs
#     obb = result.obb  # Oriented boxes object for OBB outputs
#     result.show()  # display to screen


image 1/100 d:\Research\Datasets\Kvasir-SEG\test\images\0.jpg: 640x640 1 polyp, 8.0ms
image 2/100 d:\Research\Datasets\Kvasir-SEG\test\images\1.jpg: 640x640 1 polyp, 10.0ms
image 3/100 d:\Research\Datasets\Kvasir-SEG\test\images\10.jpg: 640x640 1 polyp, 9.0ms
image 4/100 d:\Research\Datasets\Kvasir-SEG\test\images\11.jpg: 640x640 1 polyp, 8.0ms
image 5/100 d:\Research\Datasets\Kvasir-SEG\test\images\12.jpg: 640x640 1 polyp, 10.0ms
image 6/100 d:\Research\Datasets\Kvasir-SEG\test\images\13.jpg: 640x640 1 polyp, 9.0ms
image 7/100 d:\Research\Datasets\Kvasir-SEG\test\images\14.jpg: 640x640 1 polyp, 9.0ms
image 8/100 d:\Research\Datasets\Kvasir-SEG\test\images\15.jpg: 640x640 1 polyp, 10.0ms
image 9/100 d:\Research\Datasets\Kvasir-SEG\test\images\16.jpg: 640x640 1 polyp, 9.0ms
image 10/100 d:\Research\Datasets\Kvasir-SEG\test\images\17.jpg: 640x640 1 polyp, 9.0ms
image 11/100 d:\Research\Datasets\Kvasir-SEG\test\images\18.jpg: 640x640 1 polyp, 10.0ms
image 12/100 d:\Research\Datasets\Kvas

In [2]:
metrics = model.val(data="dataset-val-yolo.yaml", device="cuda")
print(metrics.box.map)  

Ultralytics 8.3.7  Python-3.11.9 torch-2.4.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1060, 6144MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning D:\Research\Datasets\Kvasir-SEG\test\labels.cache... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:09<00:00,  1.29s/it]


                   all        100        101      0.924      0.957       0.97      0.797
Speed: 3.3ms preprocess, 7.3ms inference, 0.0ms loss, 3.6ms postprocess per image
Results saved to runs\detect\val45
0.796586968020067


In [4]:
import detectron2
import cv2 as cv2
import os
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode

def get_polyp_dicts(base_dir):
    classes = ['polyp']  # Adjust class names as necessary
    image_path = os.path.join(base_dir, "images")
    annotation_path = os.path.join(base_dir, "labels")

    dataset_dicts = []

    for idx, filename in enumerate(os.listdir(annotation_path)):
        if not filename.endswith(".txt"):
            continue

        # Construct the full image path
        image_file = os.path.splitext(filename)[0] + ".jpg"  # Assuming image format is .jpg
        image_file_path = os.path.join(image_path, image_file)

        # Read image dimensions
        height, width = cv2.imread(image_file_path).shape[:2]

        # Read YOLO annotations
        with open(os.path.join(annotation_path, filename)) as f:
            lines = f.readlines()

        record = {
            "file_name": image_file_path,
            "image_id": os.path.splitext(filename)[0],  # Use image name as image_id, we will need for COCOEvaluator
            "height": height,
            "width": width,
            "annotations": []
        }

        for line in lines:
            class_id, x_center, y_center, bbox_width, bbox_height = map(float, line.strip().split())

            # Convert YOLO format to absolute coordinates
            x_center *= width
            y_center *= height
            bbox_width *= width
            bbox_height *= height

            # Calculate bounding box coordinates
            x1 = x_center - (bbox_width / 2)
            y1 = y_center - (bbox_height / 2)
            x2 = x_center + (bbox_width / 2)
            y2 = y_center + (bbox_height / 2)

            obj = {
                "bbox": [x1, y1, x2, y2],
                "bbox_mode": BoxMode.XYXY_ABS,
                "category_id": int(class_id),  # Assuming class_id is zero-based
                "iscrowd": 0
            }
            record["annotations"].append(obj)

        dataset_dicts.append(record)

    return dataset_dicts

DatasetCatalog.clear()
MetadataCatalog.clear()


classes = ['polyp']

for d in ["test"]:
    DatasetCatalog.register("polyp_" + d, lambda d=d: get_polyp_dicts(os.path.join(source, d)))
    MetadataCatalog.get("polyp_" + d).set(thing_classes=classes)

metadata = MetadataCatalog.get("polyp_test")

In [8]:
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.data import MetadataCatalog
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2 import model_zoo
import matplotlib.pyplot as plt
import cv2 as cv2
import os

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.WEIGHTS = os.path.join("model_best.pth")
predictor = DefaultPredictor(cfg)

results_detectron2 = []
for filename in os.listdir(source):
    file_path = os.path.join(source, filename)    
    im = cv2.imread(file_path)
    outputs = predictor(im)  
    # Access the 'instances' field
    instances_data = outputs['instances']

    # Retrieve pred_boxes
    # v = Visualizer(im[:, :, ::-1], metadata=metadata, scale=0.8)
    # v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    # plt.figure(figsize = (14, 10))
    # plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
    # plt.show()
    instances_data = outputs["instances"]
    boxes_np = instances_data.pred_boxes.tensor.cpu().numpy().tolist()
    scores_np = instances_data.scores.cpu().numpy().tolist()
    result = {
        'fileId': int(filename.replace(".jpg", "")),
        'boxes': boxes_np,
        'scores': scores_np,
        'categories': [1 for _ in scores_np]
    }
    results_detectron2.append(result)




c:\Users\hieut\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3610.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [7]:
import sys
import os
import warnings

warnings.filterwarnings("ignore", message="An output with one or more elements was resized since it had shape")

# Add the root directory of your project to sys.path
sys.path.append(os.path.abspath('D:\Research\ssd_pytorch'))

# Now you can import your module
from ssd_pytorch.test_default import test_voc

# Call the function
result_ssd = test_voc()




c:\Users\hieut\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\__init__.py:955: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\tensor\python_tensor.cpp:434.)
  _C._set_default_tensor_type(t)
D:\Research\ssd_pytorch\ssd.py:34: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  self.priors = Variable(self.priorbox.forward(), volatile=True)


Finished loading model!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Testing image 1/100....
D:/Research/ssd_pytorch/data/test/images\0.jpg
Testing image 2/100....
D:/Research/ssd_pytorch/data/test/images\1.jpg
Testing image 3/100....
D:/Research/ssd_pytorch/data/test/images\2.jpg
Testing image 4/100....
D:/Research/ssd_pytorch/data/test/images\3.jpg
Testing image 5/100....
D:/Research/ssd_pytorch/data/test/images\4.jpg
Testing image 6/100....
D:/Research/ssd_pytorch/data/test/images\5.jpg
Testing image 7/100....
D:/Research/ssd_pytorch/data/test/images\6.jpg
Testing image 8/100....
D:/Research/ssd_pytorch/data/test/images\7.jpg
Testing image 9/100....
D:/Research/ssd_pytorch/data/test/images\8.jpg
Testing image 10/100....
D:/Research/ssd_pytorch/data/test/images\9.jpg
Testing image 11/100....
D:/Research/ssd_pytorch/data/test/images\10.jpg
Testing image 12/100....
D:/Research/ssd_pytorch/data/test/images\11.jpg
Testing image 13/100....
D:/Research

In [4]:
def extract_ssd_bboxes(result_ssd, image_id):
    bboxes = []
    scores = []
    categories = []
    
    for item in result_ssd:
       if item['image_id'] == image_id:
            # Extract the bounding box in xywh format
            x, y, width, height = item['bbox']
            
            # Convert to xyxy format
            x_min = int(x)
            y_min = int(y)
            x_max = int(x + width)
            y_max = int(y + height)
            
            # Append the converted bounding box
            bboxes.append([x_min, y_min, x_max, y_max])
            scores.append(item['score'])
            categories.append(1)  # Assuming category ID is 1 for SSD

    return bboxes, scores, categories

def extract_detectron_bboxes(results_detectron2, image_id):
    for item in results_detectron2:
        if item['fileId'] == image_id:
            return item['boxes'], item['scores'], item['categories']
    return [], [], []

def extract_yolo_bboxes(results_yolo, image_id):
    bboxes = []
    scores = []
    categories = []
    yolo_filename = f"{image_id}.jpg"

    for result in results_yolo:
        last_component = result.path.split("\\")[-1]
        if last_component == yolo_filename:           
            boxes = result.boxes            
            xyxy = boxes.xyxy.cpu().numpy().tolist()
            conf = boxes.conf.cpu().numpy().tolist()
            if image_id == 4:
                print(boxes)
                print(xyxy)
            bboxes = xyxy
            scores = conf
            categories = [1 for _ in conf]  # Assuming category ID is 1 for YOLO
            break  # Exit loop after finding the match

    return bboxes, scores, categories

def normalize_bboxes(bboxes, image_width, image_height):
    normalized_bboxes = []
    for box_set in bboxes:
        normalized_set = []
        for box in box_set:
            x1, y1, x2, y2 = box
            normalized_box = [
                x1 / image_width,
                y1 / image_height,
                x2 / image_width,
                y2 / image_height
            ]
            normalized_set.append(normalized_box)
        normalized_bboxes.append(normalized_set)
    return normalized_bboxes

In [20]:
# For object detection annotations, the format is "bbox" : [x,y,width,height]
# Where:
# x, y: the upper-left coordinates of the bounding box
# width, height: the dimensions of your bounding box


from ensemble_boxes import *
import json

iou_thr = 0.65
skip_box_thr = 0.5
sigma = 0.0001
weights = [1, 3, 1]

def ensemble_results(result_ssd, results_detectron2, results_yolo, num_images):
    coco_results = []

    for i in range(num_images):
        bboxes_for_image = []
        scores_for_image = []
        categories_for_image = []

        # Extract boxes and scores from each model
        ssd_bboxes, ssd_scores, ssd_categories = extract_ssd_bboxes(result_ssd, i)
        detectron_bboxes, detectron_scores, detectron_categories = extract_detectron_bboxes(results_detectron2, i)
        yolo_bboxes, yolo_scores, yolo_categories = extract_yolo_bboxes(results_yolo, i)

        # Append found bounding boxes
        if ssd_bboxes:
            bboxes_for_image.append(ssd_bboxes)
            scores_for_image.append(ssd_scores)
            categories_for_image.append(ssd_categories)

        if yolo_bboxes:
            bboxes_for_image.append(yolo_bboxes)
            scores_for_image.append(yolo_scores)
            categories_for_image.append(yolo_categories)

        if detectron_bboxes:
            bboxes_for_image.append(detectron_bboxes)
            scores_for_image.append(detectron_scores)
            categories_for_image.append(detectron_categories)

        normalized_bboxes = normalize_bboxes(bboxes_for_image, 352, 352)


        # Perform Non-maximum Suppression (NMS), Soft-NMS, or Weighted Boxes Fusion (WBF)
        # You might want to choose one of these methods based on your requirement
        #boxes, scores, labels = nms(normalized_bboxes, scores_for_image, categories_for_image, weights=weights, iou_thr=iou_thr)
        #boxes, scores, labels = soft_nms(normalized_bboxes, scores_for_image, categories_for_image, weights=weights, iou_thr=iou_thr, sigma=sigma, thresh=skip_box_thr)
        boxes, scores, labels = weighted_boxes_fusion(normalized_bboxes, scores_for_image, categories_for_image, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr, conf_type='avg')
        
        if (i == 4):
            print(f"Image ID: {i}")
            print("Boxes:", bboxes_for_image)
            print("Normalized Boxes:", normalized_bboxes)
            print("Scores:", scores_for_image)
            print("Categories:", categories_for_image)
            print(boxes)
            print(scores)
            # print(labels)
            # print(filtered_boxes)
            # print(filtered_scores)
            # print(filtered_labels)
        
        for box, score, label in zip(boxes, scores, labels):
        #for box, score, label in zip(filtered_boxes, filtered_scores, filtered_labels):
            coco_result = {
                "image_id": i,
                "category_id":  int(label) if label.is_integer() else label,
                "bbox": [box[0] * 352, box[1] * 352, (box[2] - box[0]) * 352, (box[3] - box[1]) * 352],
                "score": score
            }
            coco_results.append(coco_result)
    
    output_file = 'result_ensemble.json'
    with open(output_file, 'w') as json_file:
        json.dump(coco_results, json_file, indent=4)

    print(f"Results saved to {output_file}")

    return coco_results

# Call the ensemble function
ensemble_results(result_ssd, results_detectron2, results_yolo, num_images=100)

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0.])
conf: tensor([0.9062])
data: tensor([[141.5188, 175.8475, 282.9159, 311.0783,   0.9062,   0.0000]])
id: None
is_track: False
orig_shape: (352, 352)
shape: torch.Size([1, 6])
xywh: tensor([[212.2173, 243.4629, 141.3971, 135.2308]])
xywhn: tensor([[0.6029, 0.6917, 0.4017, 0.3842]])
xyxy: tensor([[141.5188, 175.8475, 282.9159, 311.0783]])
xyxyn: tensor([[0.4020, 0.4996, 0.8037, 0.8837]])
[[141.5187530517578, 175.8475341796875, 282.9158630371094, 311.07830810546875]]
Image ID: 4
Boxes: [[[145, 179, 285, 311]], [[141.5187530517578, 175.8475341796875, 282.9158630371094, 311.07830810546875]], [[140.85691833496094, 183.54161071777344, 283.1531677246094, 327.69219970703125]]]
Normalized Boxes: [[[0.4119318181818182, 0.5085227272727273, 0.8096590909090909, 0.8835227272727273]], [[0.4020419120788574, 0.4995668584650213, 0.8037382472645153, 0.8837451934814453]], [[0.4001616998152299, 0.52142503044822, 0.8044124083085493, 0

[{'image_id': 0,
  'category_id': 1,
  'bbox': [47.21208190917969,
   31.532081604003906,
   280.71189880371094,
   312.06063079833984],
  'score': 0.5229296922683716},
 {'image_id': 1,
  'category_id': 1,
  'bbox': [81.66442108154297,
   27.696481704711914,
   179.31351470947266,
   206.15981101989746],
  'score': 0.5588735461235046},
 {'image_id': 2,
  'category_id': 1,
  'bbox': [121.17577362060547,
   85.7382583618164,
   107.85797882080078,
   122.0956497192383],
  'score': 0.5338706016540528},
 {'image_id': 4,
  'category_id': 1,
  'bbox': [141.5187530517578,
   175.8475341796875,
   141.39710998535156,
   135.23077392578125],
  'score': 0.5437288999557495},
 {'image_id': 5,
  'category_id': 1,
  'bbox': [136.3366241455078, 0.0, 146.98280334472656, 168.43682861328125],
  'score': 0.5381203293800354},
 {'image_id': 6,
  'category_id': 1,
  'bbox': [212.11000061035156,
   121.30024719238283,
   110.37666320800784,
   152.9399261474609],
  'score': 0.5324929833412171},
 {'image_id':